# Itens abordados ao longo do projeto

<div style="line-height: 2; font-size: 16px; text-align: justify; text-indent: 1em; margin: 0 0 0 10px;">
    <ol>
        <li>Algoritmo de Machine Learning (Breve explicação do seu funcionamento);</li>
        <li>Utilização do Random State;</li>
        <li>Balanceamento de dataset;</li>
        <li>Divisão do dataset (train / test);</li>
        <li>Técnicas de tratamento para Categóricas;</li>
        <li>Feature Scaling;</li>
        <li>Feature Selection;</li>
        <ul>
            <li>Identificação e eliminação de um variáveis que se comportam como ID.</li>
        </ul>
        <li>Learning Curve / Underfitting / Overfitting;</li>
        <li>Métricas para avaliação para Classificação/Regressão;</li>
        <li>Pipeline / ColumnTransformer;</li>
        <li>Otimização de Hiperparâmetros;</li>
        <li>Explicação acerca dos Hiperparâmetros escolhidos;</li>
        <li>Cross Validation;</li>
        <li>Comparativo entre mais de um algoritmo.</li>
    </ol>
</div>

# Imports